<h1> This notebook is used to create a text generation model </h1>

importing the dependecies 

In [2]:
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils

/home/arnab/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/arnab/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/arnab/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/arnab/anaconda3/lib/python3.6/site-packages/

Load the data 

In [5]:
text = (open('sonnets.txt').read())
text = text.lower()

create character/word mappings

In [6]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n,char in enumerate(characters)}
char_to_n = {char:n for n,char in enumerate(characters)}

In [7]:
n_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: "'",
 4: '(',
 5: ')',
 6: ',',
 7: '-',
 8: '.',
 9: ':',
 10: ';',
 11: '?',
 12: 'a',
 13: 'b',
 14: 'c',
 15: 'd',
 16: 'e',
 17: 'f',
 18: 'g',
 19: 'h',
 20: 'i',
 21: 'j',
 22: 'k',
 23: 'l',
 24: 'm',
 25: 'n',
 26: 'o',
 27: 'p',
 28: 'q',
 29: 'r',
 30: 's',
 31: 't',
 32: 'u',
 33: 'v',
 34: 'w',
 35: 'x',
 36: 'y',
 37: 'z'}

In [8]:
char_to_n

{'\n': 0,
 ' ': 1,
 '!': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 ':': 9,
 ';': 10,
 '?': 11,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'k': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'q': 28,
 'r': 29,
 's': 30,
 't': 31,
 'u': 32,
 'v': 33,
 'w': 34,
 'x': 35,
 'y': 36,
 'z': 37}

Data pre processing

In [9]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0,length-seq_length,1):
    sequence = text[i:i + seq_length]
    label = text[i+seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [10]:
X_modified = np.reshape(X,(len(X),seq_length,1))
Y_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [11]:
X_modified

array([[[20],
        [ 0],
        [ 0],
        ...,
        [12],
        [30],
        [ 1]],

       [[ 0],
        [ 0],
        [ 1],
        ...,
        [30],
        [ 1],
        [31]],

       [[ 0],
        [ 1],
        [17],
        ...,
        [ 1],
        [31],
        [19]],

       ...,

       [[12],
        [23],
        [23],
        ...,
        [ 1],
        [23],
        [26]],

       [[23],
        [23],
        [ 6],
        ...,
        [23],
        [26],
        [33]],

       [[23],
        [ 6],
        [ 0],
        ...,
        [26],
        [33],
        [16]]])

In [12]:
Y_modified

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

A Base line Model 

In [13]:
model = Sequential()
model.add(LSTM(400,input_shape=(X_modified.shape[1],X_modified.shape[2]),return_sequences=True))
model.add(Dropout(0,2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [14]:
model.fit(X_modified,Y_modified,epochs = 1,batch_size=100)
model.save_weights('text_generator_400_0.2_400_0.2_baseline.h5')


Epoch 1/1
97819/97819 [==============================] - 2581s 26ms/step - loss: 2.6031


Generating the text

In [17]:


string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]



In [18]:
txt=""
for char in full_string:
    txt = txt+char
txt

' the riper should by time decease,\n his tender heir might bear his memory:\n but thou, contracted to l  llllllllllllllllll  llll  lllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll'